# Classifying elliptic curves by complex multiplication

In [10]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import torch

In [11]:
df = pd.read_csv("output/dataset.csv")

In [12]:
df.head()

,A2,A3,A5,A7,A11,A13,A17,A19,A23,A29,...,A9901,A9907,A9923,A9929,A9931,A9941,A9949,A9967,A9973,CM
0,0.000000,0.577350,0.000000,0.000000,0.000000,-0.27735,0.000000,0.917663,1.668115,1.114172,...,0.532643,0.934355,0.331278,-0.541927,1.264370,-1.384089,-0.421075,-1.302150,-1.862516,0.0
1,0.707107,0.000000,0.447214,1.511858,0.000000,-1.10940,1.455214,-0.688247,-0.208514,0.000000,...,1.507481,1.547211,-1.264879,0.311106,1.314543,0.702074,1.584044,0.220364,1.361840,0.0
2,-0.707107,0.000000,-0.447214,0.000000,1.206045,-1.10940,-1.455214,0.458831,0.000000,1.856953,...,-1.286383,-0.874074,0.301162,-1.384925,-0.200694,0.140415,-1.142918,-0.030050,0.961299,0.0
3,0.000000,-1.154701,0.000000,-1.511858,-0.904534,0.55470,0.000000,0.458831,1.251086,-1.671258,...,-0.311546,1.587399,-1.415460,1.384925,-0.762636,-0.300889,1.193046,1.041720,-1.061434,0.0
4,-1.414214,0.000000,0.000000,1.133893,0.000000,-0.55470,0.727607,0.917663,0.000000,-0.928477,...,-0.693441,-0.170796,0.582246,0.511820,0.070243,-0.742193,1.162969,0.130215,1.061434,0.0


In [6]:
with open("output/dataset.csv") as f:
    lines = f.readlines()

In [8]:
len(lines[0].split(","))

10001

In [9]:
len(lines[1].split(","))

1230